# Projeto de Otimização: Minimização de Custos Logísticos

**Problema:** O objetivo deste projeto é minimizar o custo total de transporte de uma rede de distribuição.

Temos 3 Centros de Distribuição (CDs) que precisam atender a demanda de 5 Regiões. Precisamos decidir quanto produto (em unidades) enviar de cada CD para cada Região, respeitando as seguintes regras:

1.  A capacidade de envio de cada CD não pode ser excedida.
2.  A demanda total de cada Região deve ser exatamente atendida.
3.  O custo total de transporte deve ser o menor possível.

**Metodologia:**
1.  **SQL:** Extrair os dados de capacidade, demanda e custos do nosso banco de dados SQLite.
2.  **Estatística:** Analisar os dados (nesta versão, os dados são estáticos, mas em um projeto real, faríamos a previsão da demanda).
3.  **Otimização:** Usar Programação Linear (com a biblioteca PuLP) para encontrar a solução ótima.

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
from sqlalchemy import create_engine
from pulp import *

## 1. Extração de Dados (SQL)

Conectamos ao banco de dados `logistica.db` e carregamos nossas tabelas em DataFrames do Pandas.

In [2]:
# Criar a conexão com o banco SQLite
engine = create_engine('sqlite:///logistica.db')

# Usar queries SQL para carregar os dados
centros = pd.read_sql('SELECT * FROM centros', engine)
demandas = pd.read_sql('SELECT * FROM demandas', engine)
custos = pd.read_sql('SELECT * FROM custos', engine)

print("--- Centros de Distribuição (Capacidade) ---")
print(centros.to_markdown(index=False))
print("\n--- Demandas por Região ---")
print(demandas.to_markdown(index=False))
print("\n--- Matriz de Custos (CD -> Região) ---")
print(custos.to_markdown(index=False))

--- Centros de Distribuição (Capacidade) ---
| id_cd   | nome           |   capacidade_dia |
|:--------|:---------------|-----------------:|
| CD1     | Sao Paulo      |             1000 |
| CD2     | Rio de Janeiro |              800 |
| CD3     | Belo Horizonte |              700 |

--- Demandas por Região ---
| id_regiao   | nome_regiao        |   demanda_dia |
|:------------|:-------------------|--------------:|
| R1          | Sul de Minas       |           400 |
| R2          | Interior SP        |           700 |
| R3          | Capital RJ         |           500 |
| R4          | Baixada Fluminense |           300 |
| R5          | Grande BH          |           600 |

--- Matriz de Custos (CD -> Região) ---
| id_cd   | id_regiao   |   custo_por_unidade |
|:--------|:------------|--------------------:|
| CD1     | R1          |                   5 |
| CD1     | R2          |                   3 |
| CD1     | R3          |                  10 |
| CD1     | R4          |         

## 2. Análise e Preparação dos Dados

Antes de alimentar o modelo de otimização, precisamos formatar os dados. A biblioteca PuLP funciona melhor com dicionários do Python.

- `capacidade`: Um dicionário `{id_cd: capacidade_dia}`
- `demanda`: Um dicionário `{id_regiao: demanda_dia}`
- `custos_dict`: Um dicionário aninhado `{id_cd: {id_regiao: custo_por_unidade}}`

(Em um projeto real, esta seria a etapa de **Análise Estatística**, onde usaríamos técnicas de previsão (como Séries Temporais) para *estimar* os valores de `demanda_dia` em vez de usá-los como dados fixos.)

In [3]:
# 1. Listas de IDs
lista_cds = centros['id_cd'].tolist()
lista_regioes = demandas['id_regiao'].tolist()

# 2. Dicionários de dados
capacidade = centros.set_index('id_cd')['capacidade_dia'].to_dict()
demanda = demandas.set_index('id_regiao')['demanda_dia'].to_dict()
    
# 3. Dicionário aninhado para custos
# Primeiro, pivotamos o dataframe para ter CDs nas linhas e Regiões nas colunas
custos_pivot = custos.pivot(index='id_cd', columns='id_regiao', values='custo_por_unidade')

# Depois, convertemos para um dicionário aninhado
custos_dict = custos_pivot.to_dict()

# Testando os dicionários
print("Capacidades:", capacidade)
print("Demandas:", demanda)
print("Custo (ex: CD1 -> R1):", custos_dict['R1']['CD1']) # Invertido pelo pivot, corrigiremos

Capacidades: {'CD1': 1000, 'CD2': 800, 'CD3': 700}
Demandas: {'R1': 400, 'R2': 700, 'R3': 500, 'R4': 300, 'R5': 600}
Custo (ex: CD1 -> R1): 5


In [4]:
# Forma mais robusta de criar o dicionário de custos aninhado (custos[cd][regiao])
custos_dict = {}
for cd in lista_cds:
    custos_dict[cd] = {}
for idx, row in custos.iterrows():
    custos_dict[row['id_cd']][row['id_regiao']] = row['custo_por_unidade']

# Testando o dicionário corrigido
print("Custo (ex: CD1 -> R1):", custos_dict['CD1']['R1'])
print("Custo (ex: CD3 -> R5):", custos_dict['CD3']['R5'])

Custo (ex: CD1 -> R1): 5
Custo (ex: CD3 -> R5): 2


## 3. Modelo de Otimização (Programação Linear)

Agora, vamos construir o modelo com PuLP.

In [5]:
# 1. Inicializa o Problema
# Queremos MINIMIZAR, por isso 'LpMinimize'
prob = LpProblem("Minimizacao_Custos_Frete", LpMinimize)

# 2. Variáveis de Decisão
# Criamos uma variável para cada rota possível (ex: CD1 -> R1, CD1 -> R2, etc.)
# O nome da variável será 'envio_CD1_R1', por exemplo
rotas = [(cd, regiao) for cd in lista_cds for regiao in lista_regioes]
x = LpVariable.dicts("envio", (lista_cds, lista_regioes), lowBound=0, cat='Continuous')

# 3. Função Objetivo
# Queremos minimizar o Custo Total.
# Custo Total = SOMA( (custo da rota) * (quantidade enviada na rota) )
prob += lpSum([x[cd][regiao] * custos_dict[cd][regiao] for (cd, regiao) in rotas]), "Custo_Total"

# 4. Restrições

# Restrição 1: Respeitar a capacidade de cada CD
# A SOMA de tudo que SAI de um CD não pode passar de sua capacidade
for cd in lista_cds:
    prob += lpSum([x[cd][regiao] for regiao in lista_regioes]) <= capacidade[cd], f"Capacidade_CD_{cd}"

# Restrição 2: Atender 100% da demanda de cada Região
# A SOMA de tudo que CHEGA em uma Região deve ser IGUAL à sua demanda
for regiao in lista_regioes:
    prob += lpSum([x[cd][regiao] for cd in lista_cds]) == demanda[regiao], f"Demanda_Regiao_{regiao}"

# 5. Resolver o Problema
prob.solve()

1

## 4. Resultados e Conclusão

Vamos analisar a solução encontrada pelo otimizador.

In [6]:
# 1. Status da Solução
print("Status da Solução:", LpStatus[prob.status])

# 2. Custo Total Otimizado
print(f"Custo Total Ótimo = R$ {value(prob.objective):.2f}")

# 3. Plano de Envio Ótimo
print("\n--- Plano de Envio (Unidades) ---")
resultados = []
for v in prob.variables():
    if v.varValue > 0:
        # Extrai os nomes para formatar
        _, cd, regiao = v.name.split("_")
        print(f"Enviar de {cd} para {regiao}: {v.varValue} unidades")
        resultados.append({'CD': cd, 'Regiao': regiao, 'Unidades': v.varValue})

# 4. (Bônus) Verificando as capacidades
print("\n--- Utilização da Capacidade ---")
df_resultados = pd.DataFrame(resultados)
utilizacao = df_resultados.groupby('CD')['Unidades'].sum()

for cd in lista_cds:
    print(f"{cd}: Usado {utilizacao.get(cd, 0)} / {capacidade[cd]} (Capacidade)")

Status da Solução: Optimal
Custo Total Ótimo = R$ 7100.00

--- Plano de Envio (Unidades) ---
Enviar de CD1 para R1: 300.0 unidades
Enviar de CD1 para R2: 700.0 unidades
Enviar de CD2 para R3: 500.0 unidades
Enviar de CD2 para R4: 300.0 unidades
Enviar de CD3 para R1: 100.0 unidades
Enviar de CD3 para R5: 600.0 unidades

--- Utilização da Capacidade ---
CD1: Usado 1000.0 / 1000 (Capacidade)
CD2: Usado 800.0 / 800 (Capacidade)
CD3: Usado 700.0 / 700 (Capacidade)
